# PODCAST SUMMARIZER USING PYTHON  SCRIPT

Importing libraries


In [2]:
#importing libraries
import pandas as pd
import transformers
import openai
from transformers import BartTokenizer, BartForConditionalGeneration
import os

In [3]:
#specifying file path
file_path = r"116_diarized_timestamped_transcript.txt"

In [4]:
import re

def extract_topics_and_quotes(transcript_file):
    with open(transcript_file, 'r', encoding='utf-8') as file:
        transcript = file.read()

    # Replace placeholders with actual details
    guest_name = "[Guest]"
    industry = "[industry]"
    guest_background = "[Guest's Background]"
    specific_topics = "[specific topics]"

    # Extract quotes
    quotes = re.findall(r'["\'](.*?)["\']', transcript)

    # Extract major topics
    topics = re.findall(r'(### .*?)(?=\n|$)', transcript)

    # Generate shownotes
    shownotes = f"### Who is {guest_name}\n- Joe Rogan welcomes {guest_name}, a renowned {industry} known for {guest_background}. The conversation promises to delve into {specific_topics}, offering listeners a unique perspective from {guest_name}'s wealth of experience and knowledge.\n\n"

    for topic in topics:
        # Extract time stamp
        time_stamp = re.search(r'\((\d+:\d+)\)', topic).group(1)
        # Add topic heading to shownotes
        shownotes += f"{topic}\n"
        # Extract 3-4 bullet points for each topic
        for _ in range(3):
            bullet_point = input(f"Enter bullet point for {topic}: ")
            shownotes += f"- {bullet_point}\n"

    # Add memorable quotes to shownotes
    shownotes += "\n### Memorable Quotes\n"
    for quote in quotes[:5]:
        speaker = guest_name if guest_name.lower() in quote.lower() else "Joe Rogan"
        context = input(f"Enter context for the quote '{quote}': ")
        shownotes += f"- '{quote}' - {speaker} ({context})\n"

    # Write shownotes to a new file
    with open('shownotes.md', 'w', encoding='utf-8') as output_file:
        output_file.write(shownotes)

if __name__ == "__main__":
    #specifying file path
    file_path = r"116_diarized_timestamped_transcript.txt"
    extract_topics_and_quotes(file_path)
